In [1]:
# Import necessary libraries for dashboard and data processing
import os  # For file operations (e.g., checking if the logo image exists)
from jupyter_dash import JupyterDash  # Dash library for creating web apps in Jupyter Notebooks
import dash_leaflet as dl  # Dash Leaflet for interactive maps
from dash import dcc, html  # Dash components for layout and interactivity (HTML, graphs, etc.)
import plotly.express as px  # Plotly for creating visualizations
import dash_table as dt  # Dash DataTable for displaying data in table format
from dash.dependencies import Input, Output  # For creating callbacks in Dash
import base64  # For encoding images in Base64 format for inline display
import pandas as pd  # Pandas for data manipulation and analysis
from pymongo import MongoClient  # MongoDB client for database interaction
from pymongo.errors import DuplicateKeyError, ConnectionFailure  # MongoDB error handling
import logging  # For logging events for debugging and tracking

# Import the custom AnimalShelter class for interacting with MongoDB
from animal_shelter import AnimalShelter

# Initialize AnimalShelter class to connect to the MongoDB database
db = AnimalShelter()

# Fetch all records from MongoDB and convert them to a DataFrame
df = pd.DataFrame.from_records(db.read({}))  # Convert MongoDB data to DataFrame
df.drop(columns=['_id'], inplace=True)  # Drop '_id' field (not needed for display)

# Initialize the Dash app
app = JupyterDash(__name__)

# Set the browser tab title for the dashboard
app.title = "PROJECT TWO"

# Define the logo image file and encode it in Base64 if the file exists
image_filename = 'Grazioso Salvare Logo.png'
if os.path.exists(image_filename):
    with open(image_filename, 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode()  # Convert to Base64 string
else:
    encoded_image = None  # No image if the file doesn't exist

# Define the layout of the dashboard
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),  # Hidden div for internal use

    # Header section with title and subtitle
    html.Div(children=[
        html.H1('SNHU CS-340 Austin Animal Center Dashboard',  # Main title
                style={'textAlign': 'center', 'fontWeight': 'bold', 'fontFamily': 'Monospace', 'fontSize': '50px'}),
        html.H2('Project 2 - Mikayla-Joy Botha',  # Subtitle
                style={'textAlign': 'center', 'fontWeight': 'bold', 'fontFamily': 'Cursive'})
    ], style={'textAlign': 'center', 'marginTop': '20px'}),

    html.Hr(),  # Horizontal line separating the header from the content

    # Display the logo (if the file exists)
    html.Div(html.Img(id='customer-image', src=f'data:image/png;base64,{encoded_image}' if encoded_image else '',
                      alt='customer image', style={'height': '50px', 'width': '50px'}),
             style={'textAlign': 'center', 'marginTop': '20px'}),

    # Filter options for data (radio buttons)
    html.Div(dcc.RadioItems(id='filter-type', options=[
        {'label': 'Water Rescue', 'value': 'water'},
        {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
        {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
        {'label': 'Reset', 'value': 'reset'}],
        value='reset', style={'textAlign': 'center', 'marginTop': '20px'})),

    html.Hr(),  # Horizontal line separating filter section from data table

    # DataTable displaying the filtered data (Criteria #1)
    html.Div(children=[
        html.H2("Criteria # 1: Interactive Data Table",  # Title for DataTable
                style={'textAlign': 'center', 'fontFamily': 'Cursive', 'backgroundColor': 'lightpink',
                       'fontSize': '30px', 'padding': '10px'}),
        dt.DataTable(id='datatable-id', columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                     data=df.to_dict('records'), editable=False, sort_action="native", sort_mode="multi",
                     column_selectable="multi", row_selectable="multi", row_deletable=False, selected_columns=[],
                     selected_rows=[0], page_action="native", page_current=0, page_size=10)
    ], style={'marginBottom': '50px'}),  # DataTable section

    # Layout for geolocation map and graph side by side (Criteria #2 and #3)
    html.Div(children=[
        # Geolocation Chart (Criteria #2)
        html.Div(children=[
            html.H2("Criteria # 2: Geolocation Chart", style={'textAlign': 'center', 'fontFamily': 'Cursive',
                                                              'backgroundColor': 'lightpink', 'fontSize': '30px',
                                                              'padding': '10px'}),
            html.Div(id='map-id', style={'marginTop': '20px'})  # Placeholder for the map
        ], style={'flex': '1', 'padding': '20px', 'margin': '10px'}),

        # Pictograph (Criteria #3: Graph)
        html.Div(children=[
            html.H2("Criteria # 3: Graph", style={'textAlign': 'center', 'fontFamily': 'Cursive',
                                                   'backgroundColor': 'lightpink', 'fontSize': '30px',
                                                   'padding': '10px'}),
            html.Div(id='graph-id', style={'marginTop': '20px'})  # Placeholder for the graph
        ], style={'flex': '1', 'padding': '20px', 'margin': '10px'})
    ], style={'display': 'flex', 'justifyContent': 'space-between', 'alignItems': 'center', 'marginTop': '20px'})
])

# Callback to update the DataTable based on the selected filter type
@app.callback(
    Output('datatable-id', 'data'),  # Updates the table data
    Output('datatable-id', 'columns'),  # Updates the table columns
    Input('filter-type', 'value')  # Reacts to changes in the selected filter
)
def update_table(filter_type):
    """
    Updates the DataTable based on the selected filter type.
    Filters data from MongoDB based on the user-selected criteria.
    """
    # Define MongoDB queries for different filters
    if filter_type == 'water':
        query = {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                 "sex_upon_outcome": "Intact Female", "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}}
    elif filter_type == 'mount':
        query = {"animal_type": "Dog", "breed": {"$in": ["German Shepard", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                 "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}}
    elif filter_type == 'disaster':
        query = {"animal_type": "Dog", "breed": {"$in": ["Doberman Pinscher", "German Shepard", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                 "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}}
    else:  # Reset filter to show all data
        query = {}

    # Fetch filtered data from MongoDB and convert to DataFrame
    filtered_df = pd.DataFrame.from_records(db.read(query))
    filtered_df.drop(columns=['_id'], inplace=True)

    # Generate table columns and data
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in filtered_df.columns]
    data = filtered_df.to_dict('records')

    return (data, columns)  # Return updated data and columns for the table

# Callback to update the graph based on selected data in the DataTable
@app.callback(
    Output('graph-id', "children"),  # Updates the graph section
    [Input('datatable-id', "derived_virtual_data")]  # Triggered when DataTable data changes
)
def update_graphs(viewData):
    """
    Generates a graph (histogram) based on selected data in the DataTable,
    showing the frequency of different dog breeds.
    """
    if not viewData:
        return []  # Return empty if no data is selected

    # Convert selected data to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    # Create and return the histogram graph
    return [dcc.Graph(figure=px.histogram(dff, x='breed'))]

# Callback to update the map based on selected rows in the DataTable
@app.callback(
    Output('map-id', "children"),  # Updates the map section
    [Input('datatable-id', "derived_virtual_data"), Input('datatable-id', "derived_virtual_selected_rows")]  # Triggered by table data or row selection
)
def update_map(viewData, row_ids):
    """
    Updates the map based on the selected rows in the DataTable,
    displaying the location of the selected animal on the map.
    """
    if not viewData or not row_ids:
        return []  # Return empty if no data or no selected rows

    # Convert the selected data to a DataFrame and get the first selected row
    dff = pd.DataFrame.from_dict(viewData)
    row = row_ids[0]

    # Create and return the map with a marker for the selected animal's location
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[row, 13], dff.iloc[row, 14]], zoom=10,
               children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                             children=[dl.Tooltip(dff.iloc[row, 4]), dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[row, 9])])])
               ])
    ]

# Run the app in the Jupyter environment
if __name__ == "__main__":
    app.run_server(debug=True, mode='inline')


/tmp/ipykernel_661403/3930268836.py:7: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table as dt  # Dash DataTable for displaying data in table format


Database connection successful.
Dash is running on http://127.0.0.1:19334/

